In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter
import hvplot.xarray
import scipy.constants

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

import colorednoise as cn

#sys.path.append("../../preprocessing/")
#from generate_chirp import generate_chirp

sys.path.append("..")
from processing import pulse_compress

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 12})

In [ ]:
def generate_chirp_with_jitter(config, t_jitter=0):
    """
    Generate a chirp according to parameters in the config dictionary, typically
    loaded from a config YAML file.

    Returns a tuple (ts, chirp_complex), where ts is a numpy array of time
    samples, and chirp_complex is a numpy array of complex floating point values
    representing the chirp.

    If you're looking for a floating point valued chirp to use in convolution,
    this is probably the right function.

    This function does not convert the complex numpy array to the cpu format
    expected by the radar code. If you want to produce samples to feed the radar
    code, look at `generate_from_yaml_filename` (later in this file) instead.
    """
    # Load parameters
    gen_params = config["GENERATE"]
    chirp_type = gen_params["chirp_type"]
    sample_rate = gen_params["sample_rate"]
    chirp_bandwidth = gen_params["chirp_bandwidth"]
    offset = gen_params.get("lo_offset_sw", 0)
    window = gen_params["window"]
    chirp_length = gen_params["chirp_length"]
    pulse_length = gen_params.get("pulse_length", chirp_length) # default to chirp_length is no pulse_length is specified

    # Build chirp

    end_freq = chirp_bandwidth / 2 # Chirp goes from -BW/2 to BW/2
    start_freq = -1 * end_freq

    start_freq += offset
    end_freq += offset

    ts = np.arange(0, chirp_length-(1/(2*sample_rate)), 1/(sample_rate)) + t_jitter
    ts_zp = np.arange(0, (pulse_length)-(1/(2*sample_rate)), 1/(sample_rate))

    if chirp_type == 'linear':
        ph = 2*np.pi*((start_freq)*ts + (end_freq - start_freq) * ts**2 / (2*chirp_length))
    elif chirp_type == 'hyperbolic':
        ph = 2*np.pi*(-1*start_freq*end_freq*chirp_length/(end_freq-start_freq))*np.log(1- (end_freq-start_freq)*ts/(end_freq*chirp_length))
    else:
        ph = 2*np.pi*(start_freq*ts + (end_freq - start_freq) * ts**2 / (2*chirp_length))
        printf("[ERROR] Unrecognized chirp type '{chirp_type}'")
        return None, None

    chirp_complex = np.exp(1j*ph)

    if window == "blackman":
        chirp_complex = chirp_complex * np.blackman(chirp_complex.size)
    elif window == "hamming":
        chirp_complex = chirp_complex * np.hamming(chirp_complex.size)
    elif window == "kaiser14":
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 14.0)
    elif window == "kaiser10":
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 10.0)
    elif window == "kaiser18": 
        chirp_complex = chirp_complex * np.kaiser(chirp_complex.size, 18.0)
    elif window != "rectangular":
        print("[ERROR] Unrecognized window function '{window}'")
        return None, None

    chirp_complex = np.pad(chirp_complex, (int(np.floor(ts_zp.size - ts.size)/2),), 'constant')

    chirp_complex = chirp_complex

    return ts_zp, chirp_complex


In [ ]:
def plot_spectrum(ts, sig, sample_rate, one_sided=False, log_scale=False, nperseg=2**16, ax=None, label="", alpha=1.0, time_delay=None):
    if ax is None:
        fig, ax = plt.subplots()
        fig.tight_layout()
    else:
        fig = None

    # Calculate power spectral density
    freq, spectrum = scipy.signal.welch(sig, fs=sample_rate, nperseg=nperseg, return_onesided=one_sided, detrend=False, window='rectangular', scaling='density')
    
    if time_delay is not None:
        ratio = 4 * (np.sin(np.pi * freq * time_delay))**2
    
    if not one_sided:
        freq = np.fft.fftshift(freq)
        spectrum = np.fft.fftshift(spectrum)
    
    ax.plot(freq, 10*np.log10(np.abs(spectrum)), label=label, alpha=alpha)

    if time_delay is not None:
        ax.plot(freq, 10*np.log10(np.abs(ratio*spectrum)), label=f"Theoretical for delay {time_delay}", alpha=alpha, linestyle='--')

    if log_scale:
        ax.set_xscale('log')
        ax.set_xlim([0.01, None])
        # if not one_sided:
        #     raise ValueError("Can't use log scale with two-sided spectrum")
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('Power [dB]')
    ax.set_title('Frequency Domain')
    ax.grid(True)

    return fig, ax


In [ ]:
formatter_hz = EngFormatter(unit='Hz')

In [ ]:
fs = 5e6
ts = np.arange(0, 5.0, 1/fs)

noise_bandpass_center = 100000

f_zero = 10**(-20/20) * cn.powerlaw_psd_gaussian(0, len(ts))
f_zero_2 = 10**(20/20) * cn.powerlaw_psd_gaussian(0, len(ts))
# bandpass f_zero_2 to produce a much higher narrow-band noise source
sos = scipy.signal.butter(1, [0.9*noise_bandpass_center, 1.1*noise_bandpass_center], btype='bandpass', fs=fs, output='sos')
f_zero_2_filt = scipy.signal.sosfiltfilt(sos, f_zero_2)
ph_noise = f_zero_2_filt + f_zero

fig, ax = plot_spectrum(ts, ph_noise, fs, log_scale=True, one_sided=True, nperseg=len(ph_noise)/10)
ax.set_xlim([10, 1e6])
ax.set_xticks([10, 100, 1e3, 10e3, 100e3, 1e6])
ax.set_xticklabels([formatter_hz(x) for x in ax.get_xticks()])

In [ ]:
10*np.log10(np.sum(np.abs(np.exp(1j*ph_noise)**2)))

In [ ]:
config = {
    'GENERATE':
        {
            'chirp_type': 'linear',
            'sample_rate': fs,
            'chirp_bandwidth': 4e6,
            'window': "rectangular",
            'chirp_length': 20e-6,
            'pulse_length': 20e-6
        }
}
expected_chirp_length = len(np.arange(0, (config['GENERATE']['chirp_length'])-(1/(2*config['GENERATE']['sample_rate'])), 1/(config['GENERATE']['sample_rate'])))

pri = 100e-6
n_chirps = int((len(ts)) // (pri * fs))
chirps = np.zeros(shape=(n_chirps, int(fs*pri)), dtype=np.complex64)
ts_chirp, reference_chirp = generate_chirp_with_jitter(config)
print(f"n_chirps: {n_chirps}, prf: {1/pri} Hz")

delay_samples = 200

for i in range(n_chirps):
    jitter_start_idx = int(i * (pri * fs))
    # Generate chirps
    ts_chirp, chirps[i, delay_samples:delay_samples+expected_chirp_length] = generate_chirp_with_jitter(config) #, t_jitter=t_jitter[jitter_start_idx:jitter_start_idx+expected_chirp_length])
    # Add in phase contributions from TX and RX LOs
    chirps[i, delay_samples:delay_samples+expected_chirp_length] *= np.exp(1j * ph_noise[jitter_start_idx:jitter_start_idx+expected_chirp_length])


In [ ]:
n_stacks = np.append(np.logspace(0, int(np.log10(n_chirps)), 10*(int(np.log10(n_chirps))+1), dtype=int), int(n_chirps))
peaks_lin_mean = np.zeros_like(n_stacks, dtype=np.float64)
peaks_lin_std = np.zeros_like(n_stacks, dtype=np.float64)

nstack_1_peak_phases = np.zeros(shape=(n_chirps,))

for i, n_stack in enumerate(n_stacks):
    peaks_lin = []
    for start_idx in np.arange(0, n_chirps-n_stack+1, n_stack):
        stacked = np.mean(chirps[start_idx:start_idx+n_stack, :], axis=0)
        fast_time, compressed = pulse_compress(stacked, reference_chirp, fs)
        peaks_lin.append(np.max(np.abs(compressed)))
        if n_stack == 1:
            nstack_1_peak_phases[start_idx] = np.angle(compressed[np.argmax(np.abs(compressed))])
    
    peaks_lin_mean[i] = np.mean(peaks_lin)
    peaks_lin_std[i] = np.std(peaks_lin)

In [ ]:
#fig_summary, (ax_spec, ax_power) = plt.subplots(1, 2, figsize=(10,5))
#first_plot = True

In [ ]:
effective_prf = 1/(n_stacks * pri)

# Spectrum of phase noise
plot_spectrum(ts, ph_noise, fs, log_scale=True, one_sided=True, ax=ax_spec, label=formatter_hz(noise_bandpass_center), nperseg=len(ph_noise)/10)
ax_spec.set_xlim([10, 1e6])
ax_spec.set_xticks([10, 100, 1e3, 10e3, 100e3, 1e6])
ax_spec.set_xticklabels([formatter_hz(x) for x in ax_spec.get_xticks()])
ax_spec.set_ylabel('Phase Noise Spectral Density\n[10 log10(rad^2/Hz)]')
ax_spec.tick_params(axis='x', labelrotation=45)
ax_spec.set_title('')
#ax_spec.set_ylim([-10, 1])

# Peak power vs n_stack
ax_power.scatter(n_stacks, 20*np.log10(peaks_lin_mean))
ax_power.set_xscale('log')

ax_power.set_xscale('log')
#ax.set_xlim(ax.get_xlim())
x_n = np.logspace(0, int(np.log10(n_chirps)), (int(np.log10(n_chirps))+1), dtype=int)
ax_power.set_xticks(x_n)
ax_power.set_xlabel("Number of stacked chirps")
ax_power.set_xlim([1, 1000])

if first_plot:
    ax_prf = ax_power.twiny()
    first_plot = False
formatter0 = EngFormatter(unit='Hz')
#ax_prf.xaxis.set_major_formatter(formatter0)
ax_prf.set_xlabel("Effective Pulse Repetition Frequency")
ax_prf.set_xlim(1/(np.array(ax_power.get_xlim()) * pri))
ax_prf.set_xscale('log')
x_pri = 1/(x_n * pri)
ax_prf.set_xticks(x_pri)
ax_prf.set_xticklabels([f"{formatter0(x)}" for x in x_pri])
ax_prf.tick_params(axis='x', labelrotation=45)

ax_power.set_ylabel("Pulse Compressed Peak Power [dB]")

ax_power.grid('both')

ax_spec.legend()

fig_summary.tight_layout()

fig_summary

In [ ]:
fig_summary.savefig("phase_noise_intuition.png", dpi=500)